In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

cid ="cbf596c1a4d84b1e9dccdd20ec42bbb3" 
secret = "abd70f6f5e6a451a86aef3084d2a73de"

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [4]:
# timeit library to measure the time needed to run this code
import timeit
start = timeit.default_timer()

# create empty lists where the results are going to be stored
artist_name = []
track_name = []
popularity = []
track_id = []

for i in range(0,1000):
    track_results = sp.search(q='year:2019', type='track', limit=50,offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        popularity.append(t['popularity'])
      

stop = timeit.default_timer()
print ('Time to run this code (in seconds):', stop - start)

Time to run this code (in seconds): 173.49006860000003


In [5]:
print('number of elements in the track_id list:', len(track_id))

number of elements in the track_id list: 1000


In [6]:
# Load the lists in a dataframe

import pandas as pd

df_tracks = pd.DataFrame({'artist_name':artist_name,'track_name':track_name,'track_id':track_id,'popularity':popularity})
print(df_tracks.shape)
df_tracks.head()

(1000, 4)


,artist_name,track_name,track_id,popularity
0,Roddy Ricch,The Box,0nbXyq5TXYPCO7pr3N8S4I,100
1,Doja Cat,Say So,3Dv1eDb0MEgF93GpLXlucZ,98
2,SAINt JHN,Roses - Imanbek Remix,7fPuWrlpwDcHm5aHCH5D9t,88
3,Post Malone,Circles,21jGcNKet2qwijlDFuPiPb,95
4,Arizona Zervas,ROXANNE,696DnlkuDOXcMAnKlTgXXK,96


In [7]:
# group the entries by artist_name and track_name and check for duplicates

grouped = df_tracks.groupby(['artist_name','track_name'], as_index=True).size()
grouped[grouped > 1].count()

37

In [8]:
# drop the duplicates

df_tracks.drop_duplicates(subset=['artist_name','track_name'], inplace=True)

In [9]:
df_tracks.shape

(962, 4)

In [10]:
#Get audio features data


# again measuring the time
start = timeit.default_timer()

# empty list, batchsize and the counter for None results
rows = []
batchsize = 100
None_counter = 0

for i in range(0,len(df_tracks['track_id']),batchsize):
    batch = df_tracks['track_id'][i:i+batchsize]
    feature_results = sp.audio_features(batch)
    for i, t in enumerate(feature_results):
        if t == None:
            None_counter = None_counter + 1
        else:
            rows.append(t)
            
print('Number of tracks where no audio features were available:',None_counter)

stop = timeit.default_timer()
print ('Time to run this code (in seconds):',stop - start)

Number of tracks where no audio features were available: 0
Time to run this code (in seconds): 1.472783100000015


In [11]:
print('number of elements in the track_id list:', len(rows))

number of elements in the track_id list: 962


In [12]:
# Load the lists in a dataframe

df_audio_features = pd.DataFrame.from_dict(rows,orient='columns')
print("Shape of the dataset:", df_audio_features.shape)
df_audio_features.head()

Shape of the dataset: (962, 18)


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.896,0.586,10,-6.687,0,0.0559,0.1040,0.000000,0.7900,0.642,116.971,audio_features,0nbXyq5TXYPCO7pr3N8S4I,spotify:track:0nbXyq5TXYPCO7pr3N8S4I,https://api.spotify.com/v1/tracks/0nbXyq5TXYPC...,https://api.spotify.com/v1/audio-analysis/0nbX...,196653,4
1,0.787,0.673,11,-4.577,0,0.1580,0.2560,0.000004,0.0904,0.786,110.962,audio_features,3Dv1eDb0MEgF93GpLXlucZ,spotify:track:3Dv1eDb0MEgF93GpLXlucZ,https://api.spotify.com/v1/tracks/3Dv1eDb0MEgF...,https://api.spotify.com/v1/audio-analysis/3Dv1...,237893,4
2,0.784,0.721,8,-5.457,1,0.0506,0.0149,0.004240,0.2850,0.895,121.966,audio_features,7fPuWrlpwDcHm5aHCH5D9t,spotify:track:7fPuWrlpwDcHm5aHCH5D9t,https://api.spotify.com/v1/tracks/7fPuWrlpwDcH...,https://api.spotify.com/v1/audio-analysis/7fPu...,176219,4
3,0.695,0.762,0,-3.497,1,0.0395,0.1920,0.002440,0.0863,0.553,120.042,audio_features,21jGcNKet2qwijlDFuPiPb,spotify:track:21jGcNKet2qwijlDFuPiPb,https://api.spotify.com/v1/tracks/21jGcNKet2qw...,https://api.spotify.com/v1/audio-analysis/21jG...,215280,4
4,0.621,0.601,6,-5.616,0,0.1480,0.0522,0.000000,0.4600,0.457,116.735,audio_features,696DnlkuDOXcMAnKlTgXXK,spotify:track:696DnlkuDOXcMAnKlTgXXK,https://api.spotify.com/v1/tracks/696DnlkuDOXc...,https://api.spotify.com/v1/audio-analysis/696D...,163636,5


In [13]:
columns_to_drop = ['analysis_url','track_href','type','uri']
df_audio_features.drop(columns_to_drop, axis=1,inplace=True)

df_audio_features.rename(columns={'id': 'track_id'}, inplace=True)

df_audio_features.shape

(962, 14)

In [14]:
# merge both dataframes
# the 'inner' method will make sure that we only keep track IDs present in both datasets
df = pd.merge(df_tracks,df_audio_features,on='track_id',how='inner')
print("Shape of the dataset:", df_audio_features.shape)
df.head()

Shape of the dataset: (962, 14)


,artist_name,track_name,track_id,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Roddy Ricch,The Box,0nbXyq5TXYPCO7pr3N8S4I,100,0.896,0.586,10,-6.687,0,0.0559,0.1040,0.000000,0.7900,0.642,116.971,196653,4
1,Doja Cat,Say So,3Dv1eDb0MEgF93GpLXlucZ,98,0.787,0.673,11,-4.577,0,0.1580,0.2560,0.000004,0.0904,0.786,110.962,237893,4
2,SAINt JHN,Roses - Imanbek Remix,7fPuWrlpwDcHm5aHCH5D9t,88,0.784,0.721,8,-5.457,1,0.0506,0.0149,0.004240,0.2850,0.895,121.966,176219,4
3,Post Malone,Circles,21jGcNKet2qwijlDFuPiPb,95,0.695,0.762,0,-3.497,1,0.0395,0.1920,0.002440,0.0863,0.553,120.042,215280,4
4,Arizona Zervas,ROXANNE,696DnlkuDOXcMAnKlTgXXK,96,0.621,0.601,6,-5.616,0,0.1480,0.0522,0.000000,0.4600,0.457,116.735,163636,5


In [ ]:
df.to_csv('SpotifyAudioFeatures2019.csv')

In [15]:
# timeit library to measure the time needed to run this code
import timeit
start = timeit.default_timer()

# create empty lists where the results are going to be stored
artist_name = []
track_name = []
popularity = []
track_id = []

for i in range(0,3):
    track_results = sp.search(q='year:2019', type='track', limit=1,offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        popularity.append(t['popularity'])
      

stop = timeit.default_timer()
print ('Time to run this code (in seconds):', stop - start)

Time to run this code (in seconds): 3.274456199999804


In [18]:
sp.audio_features('0nbXyq5TXYPCO7pr3N8S4I')

[{'danceability': 0.896,
  'energy': 0.586,
  'key': 10,
  'loudness': -6.687,
  'mode': 0,
  'speechiness': 0.0559,
  'acousticness': 0.104,
  'instrumentalness': 0,
  'liveness': 0.79,
  'valence': 0.642,
  'tempo': 116.971,
  'type': 'audio_features',
  'id': '0nbXyq5TXYPCO7pr3N8S4I',
  'uri': 'spotify:track:0nbXyq5TXYPCO7pr3N8S4I',
  'track_href': 'https://api.spotify.com/v1/tracks/0nbXyq5TXYPCO7pr3N8S4I',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0nbXyq5TXYPCO7pr3N8S4I',
  'duration_ms': 196653,
  'time_signature': 4}]